# 2. 台車の制御とセンサ情報の取得

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>HSRの台車の制御方法と、センサ情報へのアクセス方法を学習します。</p>
</div> 

# 台車を制御する

HSRの台車は速度司令で制御することができます。
まず、この章で用いるライブラリをインポートし、初期化をします。

In [ ]:
import math
import rospy
import time
from utils import *
rospy.init_node('base_and_sensor')

台車を動かすには`move_base`という関数を使用します。興味のある人は中身を見てみてください。

```python
from geometry_msgs.msg import Twist

# 速度指令のパブリッシャーを作成
base_vel_pub = rospy.Publisher('/hsrb/command_velocity', Twist, queue_size=1)

def move_base_vel(vx, vy, vw):
    u"""台車を速度制御する関数

    引数:
        vx (float): 直進方向の速度指令値 [m/s]（前進が正、後進が負）
        vy (float): 横方向の速度指令値 [m/s]（左が正、右が負）
        vw (float): 回転方向の速度指令値 [deg/s]（左回転が正、右回転が負）

    """

    # 速度指令値をセットします
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw / 180.0 * math.pi  # 「度」から「ラジアン」に変換します
    base_vel_pub.publish(twist)  # 速度指令をパブリッシュします
```

まず、台車を前進させてみましょう。直進させたい場合は以下のようにの速度司令を入力します。ロボットが少し前進すると思います。

In [ ]:
move_base_vel(1, 0, 0)

これは、1m/秒の速度で直進することを意味します。

台車を後進させるにはマイナスの指令値を入力します。

In [ ]:
move_base_vel(-1, 0, 0)

HSRの台車は全方向に移動可能です。以下のように速度司令を入力し、左方向に移動させてみましょう。

In [ ]:
move_base_vel(0, 1, 0)

右方向に移動させるにはマイナスの指令値を入力します。

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


回転して向いている方向を変えたい場合は、以下のように制御値を設定してください。マイナスの指令値で逆回転することもできます。

In [ ]:
move_base_vel(0 , 0, 90)

台車を直進させ続けてみましょう。while文を用いることでロボットを動かし続けることができます。

In [ ]:
while True:
    move_base_vel(1, 0, 0)

壁にぶつかってしまいました。上の■ボタンを押して動作を停止させましょう。
![title](./imgs/2_stop_button.png)

一定時間動かし続けてみましょう。以下を実行すると3秒間直進させることができます。

In [ ]:
start_time = get_current_time_sec()  # 現在時刻を取得
while get_current_time_sec() - start_time < 3:  # 3秒経過後ループを抜ける
    move_base_vel(0.5, 0, 0)

0.5m/sの速度で3秒間直進させたので、約1.5m前方に進みます。

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>初期位置から人の前まで移動をするプログラムを書いてみましょう。</p>
    <p>以下のコマンドでHSRを初期位置に戻すことができます。</p>
</div>

In [ ]:
# 初期位置に戻す
move_base_goal(0, 0, 0)

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


# センサ情報を取得する
前章ではロボットの動きを人間が教えていました。次はセンサを用いて環境情報を取得して、ロボットを動かしてみましょう。

HSRには、例えば以下のセンサが搭載されています。

- レーザスキャナ：障害物までの距離を2次元的に測定


- RGB-Dカメラ：色情報(RGB)+深度情報(Depth)を測定可能なカメラ


- IMU：加速度、角加速度、磁力を測定


- エンコーダ：ロボットの各関節角度を測定

## センサ情報の確認方法

rvizを使ってセンサ情報を視覚的に確認してみましょう。

まずは、以下のコマンドを実行してrvizを起動します。

In [ ]:
%%script bash --bg
rviz -d data/2_base_and_sensor.rviz > /dev/null 2>&1

左の「Displays」の中から興味のあるデータにチェックを入れて、表示してみてください。それぞれのデータの意味は以下のとおりです。

- RobotModel: ロボットの自己位置推定、関節角度情報を反映したロボットのCGモデル


- LaserScan: レーザスキャナにより測定された障害物までの2次元距離


- Image: 頭部に搭載されたRGB-Dカメラからの映像


- PointCloud2: RGB-Dカメラから生成された環境の点群情報(ポイントクラウド)

シミュレータの様子と表示されたデータを比較してみましょう。

![title](./imgs/2_rviz_topics.png)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>HSRが搭載している各種センサ情報をrviz上で確認し、スクリーンショット、スクリーンキャストを表示してみましょう。</p>
</div>

In [ ]:
# スクリーンショットを表示
screen_shot()

In [ ]:
# スクリーンキャストを表示
screen_cast(10)

# センサ情報をプログラム上で利用する

センサ情報をプログラム上で利用してみましょう。ここでは、レーザスキャナの情報を利用してみます。

以下の「クラス」を使用します。興味のある人は見てみてください。


```python
from sensor_msgs.msg import LaserScan

class Laser():
    u"""レーザ情報を扱うクラス"""

    def __init__(self):
        # レーザースキャンのサブスクライバのコールバックに_laser_cbメソッドを登録
        self._laser_sub = rospy.Subscriber('/hsrb/base_scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb(self, msg):
        # レーザスキャンのコールバック関数
        self._scan_data = msg

    def get_data(self):
        u"""レーザの値を取得する関数"""
        return self._scan_data
```

レーザスキャナの情報を取得するために以下を実行します。

In [ ]:
laser = Laser()

以下を実行することで、データを取得することができます。取得されたセンサ値を、`scan_data`変数に格納しています。

In [ ]:
scan_data = laser.get_data()

`scan_data`変数に格納されたセンサ値の中身を見てみましょう。

In [ ]:
scan_data

センサ値は、Pythonの「構造体」を使って格納されます。

構造体の中身は「変数名.構造体の要素名」でアクセスできます。

例えば、`angle_min`, `angle_max`には、レーザスキャナのスキャン範囲（-120度から120度）がラジアンで格納されています。

In [ ]:
# scan_data.angle_maxの値をラジアンからdegに変換します
scan_data.angle_max / math.pi * 180  # math.pi = π

レーザスキャナのデータ本体は、`ranges`という名前の配列に格納されており、配列の長さは721のようです。

In [ ]:
# データの配列の長さを習得
len(scan_data.ranges)

スキャン範囲（-120度から120度）上の各観測点がデータ化されているので、配列の真ん中（=361番目）の要素が「ロボット正面から壁までの距離」を表します。

センサ値の単位はメートルです。

In [ ]:
# 361番目のデータにアクセス
scan_data.ranges[360]

# センサ値を使ってロボットを制御する

センサ値を使ってロボットを制御してみましょう。

壁の1メートル手前で止まる（壁の1メートル手前まで前進する）プログラムを書いてみましょう。

while文を使うと以下のように書けると思います。

In [ ]:
while True:
    scan_data = laser.get_data()
    if scan_data.ranges[360] < 1.0:  # 1メートル以内になったらwhileループを抜ける
        break
        
    move_base_vel(0.5, 0, 0)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>初期位置から人の前まで移動するプログラムを書いてみましょう。今回はレーザの値を用いましょう。</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


<div class="alert alert-block alert-info">
    <b>アドバンスド課題</b>
    <p>余裕がある人は「壁にぶつからないで部屋を動き回る」プログラムを書いてみましょう。</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


<div class="alert alert-block alert-info">
    <b>次の学習</b>
    <p>レーザスキャナの値と部屋の形状をマッチングすることで、ロボットの絶対位置を計算する「自己位置推定」が可能になります。</p>
    <p>HSRに搭載されている自己位置推定に基づいた高度な移動機能を使ってみましょう。</p>
</div>